In [13]:
from mxnet import nd
from mxnet.gluon import nn

## 1.继承block类来构造模型

In [14]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation = 'relu')
        self.output = nn.Dense(10)

    def forward(self, x):
        return self.output(self.hidden(x))

In [15]:
X = nd.random.uniform(shape = (2, 20))
net = MLP()
net.initialize()
net(X)


[[ 0.01901926  0.03073808  0.01856415  0.0154129  -0.05153973  0.01102755
   0.0378991   0.03151394 -0.08648601 -0.05524507]
 [ 0.06364334 -0.01114105  0.01504964 -0.0431091   0.00768508  0.04633598
   0.02869711  0.00766598 -0.05203126 -0.07329649]]
<NDArray 2x10 @cpu(0)>

## 2.Sequential类继承自Block类

In [16]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)
    
    def add(self, block):
        self._children[block.name] = block

    def forward(self, x):
        for block in self._children.values():
            x = block(x)
        return x


In [17]:
net = MySequential()
net.add(nn.Dense(256, activation = 'relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)


[[-0.07673366 -0.02770902  0.03064325 -0.03149015  0.02848469  0.10131303
   0.00625361 -0.04837742 -0.10565996 -0.07495881]
 [ 0.00015309  0.00130882 -0.0376035  -0.05839265 -0.00849809  0.06806572
  -0.01718362 -0.07539228 -0.04722349 -0.0697283 ]]
<NDArray 2x10 @cpu(0)>

## 3.构造复杂的模型

In [18]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # 使⽤get_constant创建的随机权重参数不会在训练中被迭代（即常数参数）
        self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape = (20, 20)))
        self.dense = nn.Dense(20, activation = 'relu')

    def forward(self, x):
        x = self.dense(x)

        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)

        x = self.dense(x)
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [23]:
net = FancyMLP()
net.initialize()
net(X)


[21.307877]
<NDArray 1 @cpu(0)>

In [24]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation = 'relu'),
                     nn.Dense(32, activation = 'relu'))
        self.dense = nn.Dense(16, activation ='relu')
    
    def forward(self, x):
        return self.dense(self.net(x))
net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())

net.initialize()
net(X)


[29.070877]
<NDArray 1 @cpu(0)>